In [151]:
import pandas as pd
import numpy as np
import datetime

df = pd.read_csv('../data/preprocessed/preprocessed.csv')
df_orig = df.copy()
df['StartTime'] = pd.to_datetime(df['StartTime'])
df['epoch'] = (df['StartTime'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
df_sorted_copy = df.copy().sort_values('StartTime')

In [152]:
max_epoch = 0
file_sep_list = []

for index, value in epoch_col.items():
    if index == 0:
        max_epoch = value
        file_sep_list.append([index, value])
        continue
    if value > max_epoch:
        max_epoch = value
    if value < max_epoch and value < 4000:
        file_sep_list.append([index, value])
        max_epoch = value

all_df = []
for i in range (0, len(file_sep_list)):
    if i == 0:
        all_df.append(df.iloc[:file_sep_list[i+1][0], :len(df.columns)])
    elif i + 1 == len(file_sep_list):
        all_df.append(df.iloc[file_sep_list[i][0]:len(df.index)-1, :len(df.columns)])
    
    else:
        all_df.append(df.iloc[file_sep_list[i][0]:file_sep_list[i+1][0], :len(df.columns)])


In [153]:
split_frames = []
for frames in all_df:
    split_frames.append(frames.reset_index(drop=True).sort_values('StartTime'))

In [249]:
dir_map = {
    'Dir':['   ->','   ?>','  <?>','  <->','  <-','  <?'],
    'dir_map': [1, 2, 3, 4, 5, 6]
}
dirs_df = pd.DataFrame(data=dir_map)

feature_df = df.copy().sort_values('StartTime')
feature_df = feature_df.merge(dirs_df,how='left', on=['Dir'])

In [250]:
feature_df['is_forward'] = feature_df.dir_map
feature_df['is_backward'] = feature_df.dir_map
feature_df.loc[feature_df.dir_map != 1, 'is_forward'] = 0
feature_df.loc[feature_df.dir_map != 5, 'is_backward'] = 0
feature_df.loc[feature_df.dir_map == 5, 'is_backward'] = 1

#Total Flow in forward and backward 10 min
feature_df['total_flow_f_time_10'] = feature_df[['StartTime', 'is_forward']].rolling('10T', on='StartTime').sum()['is_forward']
feature_df['total_flow_b_time_10'] = feature_df[['StartTime', 'is_backward']].rolling('10T', on='StartTime').sum()['is_backward']

In [251]:
feature_df['totbytes_f'] = feature_df.TotBytes
feature_df.loc[feature_df.is_forward == 0, 'totbytes_f'] = np.NaN

feature_df['totbytes_b'] = feature_df.TotBytes
feature_df.loc[feature_df.is_backward == 0, 'totbytes_b'] = np.NaN

forward_min_10_rolling_tot_bytes = feature_df[['StartTime', 'totbytes_f']].rolling('10T', on='StartTime')
back_min_10_rolling_tot_bytes = feature_df[['StartTime', 'totbytes_b']].rolling('10T', on='StartTime')

#Total Size in Forward and Backward 10 minutes
feature_df['sum_size_f_time_10'] = forward_min_10_rolling_tot_bytes.sum()['totbytes_f']
feature_df['sum_size_b_time_10'] = back_min_10_rolling_tot_bytes.sum()['totbytes_b']

#Min Size in Forward and Backward 10 minutes
feature_df['min_size_f_time_10'] = forward_min_10_rolling_tot_bytes.min()['totbytes_f']
feature_df['min_size_b_time_10'] = back_min_10_rolling_tot_bytes.min()['totbytes_b']

#Max Size in Forward and Backward 10 minutes
feature_df['max_size_f_time_10'] = forward_min_10_rolling_tot_bytes.max()['totbytes_f']
feature_df['max_size_b_time_10'] = back_min_10_rolling_tot_bytes.max()['totbytes_b']

#Mean Size in Forward and Backward 10 minutes
feature_df['mean_size_f_time_10'] = forward_min_10_rolling_tot_bytes.mean()['totbytes_f']
feature_df['mean_size_b_time_10'] = back_min_10_rolling_tot_bytes.mean()['totbytes_b']

#Standard Deviation Size in Forward and Backward 10 minutes
feature_df['std_size_f_time_10'] = forward_min_10_rolling_tot_bytes.std()['totbytes_f']
feature_df['std_size_b_time_10'] = back_min_10_rolling_tot_bytes.std()['totbytes_b']


In [262]:
forward_x_5_tot_bytes = feature_df[['StartTime', 'totbytes_f']].rolling(5, min_periods=1, on='StartTime')
back_x_5_tot_bytes = feature_df[['StartTime', 'totbytes_b']].rolling(5, min_periods=1, on='StartTime')

# test = feature_df[['StartTime', 'Dir', 'dir_map', 'epoch', 'TotBytes', 'is_forward']].copy()

# test[['StartTime', 'TotBytes', 'is_forward']].groupby(['is_forward']).rolling('2s', on='StartTime').sum()['TotBytes']
forward_x_5_tot_bytes.sum().head(8)

,StartTime,totbytes_f
0,1970-01-01 01:00:00.000000,4.110874
1,1970-01-01 01:00:00.000000,8.221748
2,1970-01-01 01:00:00.000000,12.332622
3,1970-01-01 01:00:00.000000,16.443495
4,1970-01-01 01:00:06.524077,25.570346
5,1970-01-01 01:00:06.653993,25.828920
6,1970-01-01 01:00:06.654046,27.069904
7,1970-01-01 01:00:06.654282,28.157527


In [263]:
feature_df[['StartTime', 'totbytes_f']].head(8)

,StartTime,totbytes_f
0,1970-01-01 01:00:00.000000,4.110874
1,1970-01-01 01:00:00.000000,4.110874
2,1970-01-01 01:00:00.000000,4.110874
3,1970-01-01 01:00:00.000000,4.110874
4,1970-01-01 01:00:06.524077,9.126850
5,1970-01-01 01:00:06.653993,4.369448
6,1970-01-01 01:00:06.654046,5.351858
7,1970-01-01 01:00:06.654282,5.198497


In [252]:
feature_df[['TotBytes', 
            'min_size_f_time_10', 
            'min_size_b_time_10', 
            'max_size_f_time_10',
            'max_size_b_time_10',
            'mean_size_f_time_10',
            'mean_size_b_time_10',
            'std_size_f_time_10',
            'std_size_b_time_10',
            'epoch']].head(25)

,TotBytes,min_size_f_time_10,min_size_b_time_10,max_size_f_time_10,max_size_b_time_10,mean_size_f_time_10,mean_size_b_time_10,std_size_f_time_10,std_size_b_time_10,epoch
0,4.110874,4.110874,NaN,4.110874,NaN,4.110874,NaN,NaN,NaN,3600
1,4.110874,4.110874,NaN,4.110874,NaN,4.110874,NaN,0.000000,NaN,3600
2,4.110874,4.110874,NaN,4.110874,NaN,4.110874,NaN,0.000000,NaN,3600
3,4.110874,4.110874,NaN,4.110874,NaN,4.110874,NaN,0.000000,NaN,3600
4,9.126850,4.110874,NaN,9.126850,NaN,5.114069,NaN,2.243213,NaN,3606
5,4.369448,4.110874,NaN,9.126850,NaN,4.989966,NaN,2.029289,NaN,3606
6,5.351858,4.110874,NaN,9.126850,NaN,5.041665,NaN,1.857522,NaN,3606
7,5.198497,4.110874,NaN,9.126850,NaN,5.061269,NaN,1.720625,NaN,3606
8,6.388561,4.110874,NaN,9.126850,NaN,5.208746,NaN,1.669199,NaN,3606
9,9.126850,4.110874,NaN,9.126850,NaN,5.600556,NaN,2.002948,NaN,3606


In [237]:
test_map = {
    'Dir':[np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN],
    'dir_map': [1, 2, 3, 4, 5, 6]
}
test_df = pd.DataFrame(data=test_map)

In [238]:
test_df

,Dir,dir_map
0,NaN,1
1,NaN,2
2,NaN,3
3,NaN,4
4,NaN,5
5,NaN,6


In [239]:
test_df.min()

Dir        NaN
dir_map    1.0
dtype: float64